# Image Classification CNN - PyTorch

## Imports

In [27]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary

import torchvision
import torchvision.transforms as transforms

In [28]:
import ssl
import certifi

# Use certifi's CA bundle for HTTPS (fixes SSL verification on macOS)
ssl._create_default_https_context = lambda: ssl.create_default_context(
    cafile=certifi.where()
)

In [29]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)

## Data

In [30]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

In [31]:
train_data = torchvision.datasets.CIFAR10(
    root="../data", train=True, transform=transform, download=True
)
test_data = torchvision.datasets.CIFAR10(
    root="../data", train=False, transform=transform, download=True
)

train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=32, shuffle=True, num_workers=2
)
test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=32, shuffle=False, num_workers=2
)

/Users/albertopuliga/Desktop/Coding/CIFAR10-Pytorch/.venv/lib/python3.11/site-packages/torchvision/datasets/cifar.py:83: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  entry = pickle.load(f, encoding="latin1")


In [32]:
image, label = train_data[0]
print(image.shape)

torch.Size([3, 32, 32])


In [33]:
class_names = [
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

## Model

In [34]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 12, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 24, 5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.flatten(1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [35]:
net = NeuralNet()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

summary(
    net,
    input_size=(1, 3, 32, 32),
    col_names=["input_size", "output_size", "num_params"],
)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
NeuralNet                                [1, 3, 32, 32]            [1, 10]                   --
├─Conv2d: 1-1                            [1, 3, 32, 32]            [1, 12, 28, 28]           912
├─MaxPool2d: 1-2                         [1, 12, 28, 28]           [1, 12, 14, 14]           --
├─Conv2d: 1-3                            [1, 12, 14, 14]           [1, 24, 10, 10]           7,224
├─MaxPool2d: 1-4                         [1, 24, 10, 10]           [1, 24, 5, 5]             --
├─Linear: 1-5                            [1, 600]                  [1, 120]                  72,120
├─Linear: 1-6                            [1, 120]                  [1, 84]                   10,164
├─Linear: 1-7                            [1, 84]                   [1, 10]                   850
Total params: 91,270
Trainable params: 91,270
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 1.52
Input si

## Train

In [36]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

net.to(device)

EPOCHS = 30
for epoch in range(EPOCHS):
    print("Training epoch: ", epoch)

    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Loss: {running_loss / len(train_loader):.4f}")

Training epoch:  0
Loss: 2.1500
Training epoch:  1
Loss: 1.7059
Training epoch:  2
Loss: 1.5049
Training epoch:  3
Loss: 1.3849
Training epoch:  4
Loss: 1.2843
Training epoch:  5
Loss: 1.1973
Training epoch:  6
Loss: 1.1194
Training epoch:  7
Loss: 1.0627
Training epoch:  8
Loss: 1.0096
Training epoch:  9
Loss: 0.9645
Training epoch:  10
Loss: 0.9254
Training epoch:  11
Loss: 0.8875
Training epoch:  12
Loss: 0.8571
Training epoch:  13
Loss: 0.8261
Training epoch:  14
Loss: 0.7979
Training epoch:  15
Loss: 0.7676
Training epoch:  16
Loss: 0.7384
Training epoch:  17
Loss: 0.7122
Training epoch:  18
Loss: 0.6896
Training epoch:  19
Loss: 0.6637
Training epoch:  20
Loss: 0.6424
Training epoch:  21
Loss: 0.6178
Training epoch:  22
Loss: 0.5959
Training epoch:  23
Loss: 0.5735
Training epoch:  24
Loss: 0.5548
Training epoch:  25
Loss: 0.5333
Training epoch:  26
Loss: 0.5119
Training epoch:  27
Loss: 0.4923
Training epoch:  28
Loss: 0.4707
Training epoch:  29
Loss: 0.4527


In [37]:
torch.save(net.state_dict(), "model.pth")

In [38]:
net = NeuralNet()
net.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

## Evaluation

In [39]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")

Accuracy: 69.58%
